In [118]:
class Subset:
    
    def __init__(self, subset, weight, ID):
        self.subset = subset
        self.weight = weight
        self.ID = ID
        
    def get_subset(self):
        return self.subset
    
    def get_weight(self):
        return self.weight
    
    def get_ID(self):
        return self.ID
    
    def contains(self, n):
        contains = False
        for m in subset:
            if m == n:
                contains = True
        return contains

    def get_size(self):
        return len(self.subset)
    
    def to_string(self):
        output = ""
        for n in self.subset:
            output = output+str(n)+" "
        output = output + "\n"+str(self.weight)
        return output

In [130]:
class ALGOBowl:
    
    def __init__(self):
        self.n=0
        self.m=0
        self.subsets = []

    def read_input(self, filename):
        file = open(filename,'r')
        self.n = int(file.readline())
        self.m = int(file.readline())
        for i in range(1,self.m+1):
            temp = []
            for string in file.readline().split(" "):
                if(string == "\n"):
                    continue
                else:
                    temp.append(int(string))
            weight = int(file.readline())
            ss = Subset(temp,weight,i)
            self.subsets.append(ss)
    
    def display_data(self):
        print(self.n)
        print(self.m)
        for s in self.subsets:
            print(s.to_string())

# not sure if usefull
#     def get_subsets():
#         sets = []
#         for s in subsets:
#             sets.append(s.get_subset())
#         return sets
    
    def disjoint(self, *args, sets = None):
        if sets == None:
            nums = set()
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
        else:
            nums = set()
            for s in sets:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
   
    def run_most_disjoint(self):
        IDs = []
        solution = []
        total_weight=0
        nums = set()
        for s in self.subsets:
            if self.disjoint(s.get_subset(), sets = solution):
                solution.append(s.get_subset())
                IDs.append(s.get_ID())
                total_weight = total_weight + s.get_weight()
                nums.update(s.get_subset())
        if len(nums) != self.n:
            rest = self.find_needed_subsets(nums)
            for ids in rest[0]:
                IDs.append(ids)
            total_weight = total_weight + rest[1]
            for s in rest[2]:
                solution.append(s)
        return [IDs, total_weight, solution]


    def find_needed_subsets(self, nums):
        IDs = []
        solution = []
        total_weight =0
        needed_numbers = set()
        for n in range(1,self.n+1):
            if not n in nums:
                needed_numbers.add(n)
        for n in needed_numbers:
            if n in nums:
                continue
            for s in self.subsets:
                for m in s.get_subset():
                    if n == m:
                        IDs.append(s.get_ID())
                        solution.append(s.get_subset())
                        total_weight = total_weight + s.get_weight()
                        nums.add(n)
                        break
        return [IDs, total_weight, solution]
                               
        
    def print_solution(self, solution):
        output = str(solution[1])+"\n"
        for s in solution[0]:
            output = output + str(s) + " "
        return output
    
            

In [131]:
#main
test = ALGOBowl()
test.read_input("input_100.txt")
# test.display_data()
print(test.print_solution(test.run_most_disjoint()))

62736
1 13 33 42 3 6 7 9 12 14 16 18 23 30 31 32 34 35 37 41 44 2 7 12 21 23 26 27 28 31 35 36 37 41 43 49 50 6 12 14 18 21 25 28 30 32 34 35 36 41 44 46 47 50 2 9 11 18 19 21 23 25 38 40 44 50 5 9 10 14 16 20 21 25 27 30 32 34 37 41 43 44 45 46 47 2 4 10 11 16 21 25 31 41 43 45 46 2 5 9 10 11 12 14 15 16 20 21 22 23 25 27 30 31 32 35 37 40 45 47 2 8 9 10 14 16 18 21 22 23 26 27 35 36 39 41 44 46 9 10 11 16 17 19 21 23 25 26 27 30 31 36 43 46 47 49 7 9 10 12 14 16 23 26 29 31 32 35 37 41 44 45 46 2 7 9 10 12 14 15 16 21 23 26 27 30 31 32 35 37 39 44 46 50 9 10 11 12 14 15 18 21 27 32 34 35 37 39 46 47 7 10 11 14 18 21 23 25 27 28 31 32 34 35 37 38 39 40 41 43 46 47 2 3 5 6 7 8 9 14 15 16 17 19 21 23 27 28 30 32 34 35 41 43 46 47 48 7 8 10 12 14 15 16 18 25 27 30 35 36 37 41 43 46 47 50 2 9 15 21 23 26 27 28 30 31 32 35 40 41 45 46 47 9 10 12 15 21 25 27 34 35 37 41 43 45 47 2 7 9 10 14 18 21 28 32 37 39 41 44 46 47 48 3 8 9 12 14 19 23 30 31 37 40 41 45 46 47 49 2 12 15 18 20 23 25 28 